In [1]:
# If you run this notebook on Google Colaboratory, uncomment the below to install automl_alex.
#!pip install -q -U automl_alex

In [1]:
import automl_alex
import sklearn
import pandas as pd
import time
from automl_alex import DataPrepare
from automl_alex import AutoML, AutoMLClassifier, AutoMLRegressor
print('AutoML-Alex version:', automl_alex.__version__)

AutoML-Alex version: 1.3.5


In [2]:
RANDOM_SEED = 42

# Classifier

## Data

In [3]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
# https://www.openml.org/d/179
dataset = fetch_openml(data_id=179, as_frame=True)
dataset.target = dataset.target.astype('category').cat.codes
dataset.data.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native-country
0,2,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,1,0,2,United-States
1,3,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,0,United-States
2,2,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,2,United-States
3,3,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,2,United-States
4,1,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,2,Cuba


In [4]:
X_train, X_test, y_train, y_test = train_test_split(dataset.data, 
                                                    dataset.target,
                                                    test_size=0.25, 
                                                    random_state=RANDOM_SEED,)
X_train.shape, X_test.shape

((36631, 14), (12211, 14))

## AutoML

In [5]:
model = AutoMLClassifier(random_state=RANDOM_SEED,)
model.fit(X_train, y_train, timeout=600)

01:23:34 | > Start Fit Base Model
01:24:15 | > DATA PREPROC
01:24:15 | Source data shape: (36631, 14)
01:24:15 | ##################################################
01:24:15 | ! START preprocessing Data
01:24:15 | - Auto detect cat features: 12
01:24:15 | > Binary Features
01:24:15 | > Clean Categorical Features
01:24:16 | > Transform Categorical Features.
01:24:16 |  - Encoder: OneHotEncoder ADD features: 135
01:24:16 |  - Encoder: CountEncoder ADD features: 12
01:24:16 |   No nans features
01:24:16 | > CleanOutliers
01:24:16 | Num of outlier detected: 527 in Feature fnlwgt
01:24:16 | Proportion of outlier detected: 1.4 %
01:24:16 | Num of outlier detected: 231 in Feature education-num
01:24:16 | Proportion of outlier detected: 0.6 %
01:24:17 | > Generate interaction Num Features
01:24:17 |  ADD features: 5
01:24:17 | > Normalization Features
01:24:17 | ##################################################
01:24:17 | Final data shape: (36631, 168)
01:24:17 | Total ADD columns: 154
01:24:1

In [6]:
predicts = model.predict(X_test)

Finished loading model, total used 300 iterations
Finished loading model, total used 300 iterations
Finished loading model, total used 300 iterations
Finished loading model, total used 300 iterations
Finished loading model, total used 300 iterations
Finished loading model, total used 300 iterations
Finished loading model, total used 300 iterations
Load CrossValidation
Load model


In [7]:
print('Test AUC: ', round(sklearn.metrics.roc_auc_score(y_test, predicts),4))

Test AUC:  0.9137


## Save & Load

In [8]:
model.save('AutoML_model_1')

Save model


In [9]:
model_new = AutoMLClassifier(random_state=RANDOM_SEED,)
model_new = model_new.load('AutoML_model_1')

01:40:22 | Load DataPrepare
01:40:22 | Load DataPrepare
01:40:22 | Load Model
Finished loading model, total used 300 iterations
01:40:22 | Load Model
Finished loading model, total used 300 iterations
01:40:22 | Load Model
Finished loading model, total used 300 iterations
01:40:22 | Load Model
Finished loading model, total used 300 iterations
01:40:22 | Load Model
Finished loading model, total used 300 iterations
01:40:23 | Load Model
Finished loading model, total used 300 iterations
01:40:23 | Load Model
Finished loading model, total used 300 iterations
Load CrossValidation
Load model
01:40:23 | Load AutoML


In [10]:
predicts = model_new.predict(X_test)
print('Test AUC: ', round(sklearn.metrics.roc_auc_score(y_test, predicts),4))

Finished loading model, total used 300 iterations
Finished loading model, total used 300 iterations
Finished loading model, total used 300 iterations
Finished loading model, total used 300 iterations
Finished loading model, total used 300 iterations
Finished loading model, total used 300 iterations
Finished loading model, total used 300 iterations
Load CrossValidation
Load model
Test AUC:  0.9131


# Regression

## Data

In [11]:
# https://www.openml.org/d/543
dataset = fetch_openml(data_id=543, as_frame=True)

X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(dataset.data), 
                                                    pd.DataFrame(dataset.target), 
                                                    test_size=0.15, 
                                                    random_state=RANDOM_SEED,)

X_train.shape, X_test.shape

((430, 19), (76, 19))

In [12]:
X_train.head(5)

,TOWN,TOWN_ID,TRACT,LON,LAT,MEDV,CMEDV,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B
104,Medford,24.0,3395.0,-71.0690,42.2480,20.1,20.1,0.13960,0.0,8.56,0,0.5200,6.167,90.0,2.4210,5,384.0,20.9,392.69
203,Weston,37.0,3671.0,-71.1990,42.2320,48.5,48.5,0.03510,95.0,2.68,0,0.4161,7.853,33.2,5.1180,4,224.0,14.7,392.78
381,Boston_East_Boston,79.0,407.0,-71.0410,42.2290,10.9,10.9,15.87440,0.0,18.10,0,0.6710,6.545,99.1,1.5192,24,666.0,20.2,396.90
489,Chelsea,89.0,1602.0,-71.0228,42.2335,7.0,7.0,0.18337,0.0,27.74,0,0.6090,5.414,98.3,1.7554,4,711.0,20.1,344.05
69,Wilmington,16.0,3313.0,-71.1110,42.3270,20.9,20.9,0.12816,12.5,6.07,0,0.4090,5.885,33.0,6.4980,4,345.0,18.9,396.90


## AutoML

In [13]:
model = AutoMLRegressor(random_state=RANDOM_SEED,)
model.fit(X_train, y_train, timeout=900, verbose=1)

01:40:29 | Source data shape: (430, 19)
01:40:29 | ##################################################
01:40:29 | ! START preprocessing Data
01:40:29 | > Binary Features
01:40:29 | > Clean Categorical Features
01:40:29 |   No nans features
01:40:29 | > CleanOutliers
01:40:29 | Num of outlier detected: 1 in Feature DIS
01:40:29 | Proportion of outlier detected: 0.2 %
01:40:30 | Num of outlier detected: 49 in Feature ZN
01:40:30 | Proportion of outlier detected: 11.4 %
01:40:30 | Num of outlier detected: 15 in Feature RM
01:40:30 | Proportion of outlier detected: 3.5 %
01:40:30 | Num of outlier detected: 27 in Feature CMEDV
01:40:30 | Proportion of outlier detected: 6.3 %
01:40:30 | Num of outlier detected: 47 in Feature CRIM
01:40:30 | Proportion of outlier detected: 10.9 %
01:40:30 | Num of outlier detected: 26 in Feature MEDV
01:40:30 | Proportion of outlier detected: 6.0 %
01:40:30 | Num of outlier detected: 60 in Feature B
01:40:30 | Proportion of outlier detected: 14.0 %
01:40:30 | 

In [14]:
predicts = model.predict(X_test)
print('Test MSE: ', round(sklearn.metrics.mean_squared_error(y_test, predicts),4))

Finished loading model, total used 300 iterations
Finished loading model, total used 300 iterations
Finished loading model, total used 300 iterations
Finished loading model, total used 300 iterations
Finished loading model, total used 300 iterations
Finished loading model, total used 300 iterations
Finished loading model, total used 300 iterations
Load CrossValidation
Load model
Test MSE:  6.8811
